<a href="https://colab.research.google.com/github/Chuc-ngan/Low-Light-Image-Enhancement/blob/main/Wavelet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==2.18.0  # Thay đổi phiên bản theo nhu cầu

# **1. Kết nối với Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/MyDrive/ColabNotebooks/TieuLuan'

!pwd

Mounted at /content/gdrive
/content/gdrive/.shortcut-targets-by-id/1DnsWp1mopgehQJUYpN4ERfnj2CyapJGW/ColabNotebooks/TieuLuan
/content/gdrive/.shortcut-targets-by-id/1DnsWp1mopgehQJUYpN4ERfnj2CyapJGW/ColabNotebooks/TieuLuan


# **2. Import thư viện**

In [ ]:
# ===================== 1. Xử lý dữ liệu =====================
import os
import shutil  # Quản lý tệp/thư mục
import random
import math
import collections
import pywt
import numpy as np  # Xử lý số học
import pandas as pd  # Xử lý dữ liệu CSV
from pathlib import Path
from PIL import Image, UnidentifiedImageError
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm
import pickle
# ===================== 2. Machine Learning =====================
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import seaborn as sns

# ===================== 3. TensorFlow & Keras =====================
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import mixed_precision
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau

# ===================== 4. Xây dựng mô hình =====================
from tensorflow.keras.layers import (
    Input, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization,
    GlobalAveragePooling2D, Dense, Dropout, RepeatVector, Reshape, Flatten
)
from tensorflow.keras.applications import EfficientNetB0, ResNet50
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input, decode_predictions
from keras.layers import add, concatenate
from keras import layers, backend as K

# ===================== 5. Xử lý hình ảnh & Vẽ biểu đồ =====================
import cv2 as cv
import matplotlib.pyplot as plt

# ===================== 6. Đặt Seed (Reproducibility) =====================
tf.random.set_seed(42)  # Thay số 42 bằng giá trị seed mong muốn
np.random.seed(1)


# **3. Cấu hình môi trường TensorFlow**

In [ ]:
# Thiết lập GPU sử dụng bộ nhớ động (tránh lỗi thiếu bộ nhớ)
gpu_devices = tf.config.list_physical_devices('GPU')
if gpu_devices:
    tf.config.experimental.set_memory_growth(gpu_devices[0], True)

# Kiểm tra TensorFlow đang chạy trên GPU hay không
print("TensorFlow is running on GPU:", tf.test.is_built_with_cuda())
# Bật chế độ Mixed Precision để tăng tốc (nếu GPU hỗ trợ)
# Thiết lập mixed precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

# Bật XLA với cấu hình tối ưu
tf.config.optimizer.set_jit(True)
tf.config.optimizer.set_experimental_options({
    "layout_optimizer": True,
    "constant_folding": True,
    "shape_optimization": True,
    "remapping": True,
    "arithmetic_optimization": True,
    "dependency_optimization": True,
    "loop_optimization": True,
    "function_optimization": True,
    "debug_stripper": True
})

TensorFlow is running on GPU: True


# **4. Configuration**

In [ ]:
# Thư mục thiếu sáng
low_light_dir = "dataset/low_light"

low_light_noise_dir = "dataset/low_light_noise"

# Ảnh đã tăng cường bằng phương pháp Wavelet
enhanced_wavelet_low_light_dir = "dataset/enhanced_wavelet_low_light"
enhanced_wavelet_low_light_noise_dir = "dataset/enhanced_wavelet_low_light_noise"

# **5. Wavelet**

In [ ]:
# Hàm xử lý một kênh ảnh (Y) bằng Wavelet
def enhance_wavelet_channel(channel, threshold=5, amplify_factor=6.0, ll_factor=1.8, denoise_thresh=10):
    # Lấy kích thước ảnh
    h, w = channel.shape
    # Đảm bảo chiều cao chẵn để tránh lỗi DWT
    if h % 2 != 0:
        channel = channel[:-1, :]  # Cắt bỏ hàng cuối nếu chiều cao lẻ
    # Đảm bảo chiều rộng chẵn để tránh lỗi DWT
    if w % 2 != 0:
        channel = channel[:, :-1]  # Cắt bỏ cột cuối nếu chiều rộng lẻ

    # Phân tích Wavelet rời rạc (DWT2) với wavelet sym5
    coeffs = pywt.dwt2(channel, 'sym5')  # Tách ảnh thành LL (độ sáng) và LH, HL, HH (chi tiết tần số cao)
    LL, (LH, HL, HH) = coeffs  # Gán các thành phần Wavelet

    # Khử nhiễu nhẹ bằng soft thresholding cho các thành phần tần số cao
    LH = pywt.threshold(LH, denoise_thresh, mode='soft')  # Khử nhiễu cho LH với ngưỡng denoise_thresh=10
    HL = pywt.threshold(HL, denoise_thresh, mode='soft')  # Khử nhiễu cho HL
    HH = pywt.threshold(HH, denoise_thresh, mode='soft')  # Khử nhiễu cho HH

    # Khuếch đại chi tiết tần số cao nếu giá trị tuyệt đối vượt ngưỡng
    LH = np.where(np.abs(LH) > threshold, LH * amplify_factor, LH)  # Khuếch đại LH với amplify_factor=6.0 nếu > threshold=5
    HL = np.where(np.abs(HL) > threshold, HL * amplify_factor, HL)  # Khuếch đại HL
    HH = np.where(np.abs(HH) > threshold, HH * amplify_factor, HH)  # Khuếch đại HH

    # Khuếch đại thành phần LL để tăng độ sáng toàn ảnh
    LL = LL * ll_factor  # Nhân LL với ll_factor=1.8 để tăng độ sáng

    # Tái tạo ảnh từ các hệ số Wavelet đã chỉnh sửa
    enhanced = pywt.idwt2((LL, (LH, HL, HH)), 'sym5')  # Sử dụng biến đổi Wavelet ngược (IDWT2) với sym5

    # Cắt ảnh về kích thước gốc nếu đã bị cắt trước đó
    enhanced = enhanced[:h, :w]  # Đảm bảo kích thước đúng với ảnh ban đầu

    # Giới hạn giá trị pixel trong [0, 255] và chuyển về định dạng uint8
    return np.clip(enhanced, 0, 255).astype(np.uint8)

# Hàm xử lý toàn bộ ảnh bằng Wavelet
def enhance_wavelet(img):
    # Chuyển ảnh từ không gian màu BGR sang YCrCb
    ycrcb = cv.cvtColor(img, cv.COLOR_BGR2YCrCb)  # Tách kênh Y (độ sáng) và Cr, Cb (màu sắc)
    y, cr, cb = cv.split(ycrcb)  # Tách các kênh Y, Cr, Cb

    # Áp dụng cải thiện Wavelet cho kênh Y
    y_enhanced = enhance_wavelet_channel(y)  # Xử lý kênh độ sáng bằng Wavelet

    # Gộp lại các kênh Y đã cải thiện với Cr, Cb
    enhanced_ycrcb = cv.merge([y_enhanced, cr, cb])  # Tạo ảnh YCrCb từ các kênh đã xử lý

    # Chuyển ảnh từ YCrCb về không gian màu BGR
    enhanced_bgr = cv.cvtColor(enhanced_ycrcb, cv.COLOR_YCrCb2BGR)  # Chuyển về BGR để lưu hoặc hiển thị

    return enhanced_bgr  # Trả về ảnh đã cải thiện

# Hàm xử lý hàng loạt ảnh bằng Wavelet
def enhance_with_wavelet(input_dir, output_dir, img_size=(224, 224)):
    os.makedirs(output_dir, exist_ok=True)  # Tạo thư mục đầu ra nếu chưa tồn tại

    all_images = []  # Danh sách lưu cặp đường dẫn ảnh đầu vào và đầu ra
    for root, _, files in os.walk(input_dir):  # Duyệt qua tất cả file trong thư mục đầu vào
        rel_path = os.path.relpath(root, input_dir)  # Tính đường dẫn tương đối
        save_dir = os.path.join(output_dir, rel_path)  # Tạo thư mục đầu ra tương ứng
        os.makedirs(save_dir, exist_ok=True)  # Tạo thư mục nếu chưa tồn tại

        # Lọc các file ảnh hợp lệ (.jpg, .png, .jpeg)
        for file in files:
            if file.lower().endswith((".jpg", ".png", ".jpeg")):
                all_images.append((os.path.join(root, file), os.path.join(save_dir, file)))  # Thêm cặp (đầu vào, đầu ra)

    # Xử lý từng ảnh với tiến độ hiển thị
    with tqdm(total=len(all_images), desc="Enhancing with Wavelet", ncols=100) as pbar:
        for img_path, save_path in all_images:  # Duyệt qua từng cặp ảnh
            img = cv.imread(img_path)  # Đọc ảnh đầu vào
            img = cv.resize(img, img_size)  # Thay đổi kích thước ảnh về 224x224

            enhanced_img = enhance_wavelet(img)  # Áp dụng cải thiện Wavelet cho ảnh

            # Lưu ảnh cải thiện với chất lượng JPEG 95
            cv.imwrite(save_path, enhanced_img, [cv.IMWRITE_JPEG_QUALITY, 95])  # Lưu ảnh vào thư mục đầu ra
            pbar.update(1)  # Cập nhật thanh tiến độ

    # In thông báo hoàn thành
    print(f"✅ Đã lưu {len(all_images)} ảnh sau khi áp dụng Wavelet vào: {output_dir}")

In [ ]:
enhance_with_wavelet(low_light_dir, enhanced_wavelet_low_light_dir)

Enhancing with Wavelet: 100%|█████████████████████████████████| 10402/10402 [32:40<00:00,  5.30it/s]

✅ Đã lưu 10402 ảnh sau khi áp dụng Wavelet vào: dataset/enhanced_wavelet_low_light


In [ ]:
enhance_with_wavelet(low_light_noise_dir, enhanced_wavelet_low_light_noise_dir)

Enhancing with Wavelet: 100%|███████████████████████████████| 10402/10402 [1:02:39<00:00,  2.77it/s]

✅ Đã lưu 10402 ảnh sau khi áp dụng Wavelet vào: dataset/enhanced_wavelet_low_light_noise
